In [1]:
!pip install azure-storage-blob

     -------------------------------------- 394.5/394.5 kB 3.1 MB/s eta 0:00:00
     ---------------------------------------- 2.9/2.9 MB 23.0 MB/s eta 0:00:00
     -------------------------------------- 193.4/193.4 kB 2.3 MB/s eta 0:00:00
     ---------------------------------------- 41.7/41.7 kB ? eta 0:00:00
     ---------------------------------------- 62.6/62.6 kB 1.1 MB/s eta 0:00:00
     ---------------------------------------- 181.6/181.6 kB ? eta 0:00:00
     ------------------------------------ 117.6/117.6 kB 761.1 kB/s eta 0:00:00
     ------------------------------------ 100.3/100.3 kB 819.1 kB/s eta 0:00:00
     ---------------------------------------- 163.8/163.8 kB ? eta 0:00:00
     ---------------------------------------- 61.6/61.6 kB 3.4 MB/s eta 0:00:00
     -------------------------------------- 121.1/121.1 kB 2.4 MB/s eta 0:00:00



[notice] A new release of pip available: 22.2.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
!pip install pandas

     --------------------------------------- 11.6/11.6 MB 38.6 MB/s eta 0:00:00
     --------------------------------------- 15.8/15.8 MB 38.5 MB/s eta 0:00:00
     ------------------------------------- 505.5/505.5 kB 33.0 MB/s eta 0:00:00
     -------------------------------------- 345.4/345.4 kB 3.0 MB/s eta 0:00:00



[notice] A new release of pip available: 22.2.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:

from io import StringIO
import pandas as pd
import numpy as np
import json
import requests
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient

In [3]:
# Load the dataset
url = "https://data.cdc.gov/api/views/hksd-2xuw/rows.csv?accessType=DOWNLOAD"

# Use pandas to read the CSV directly from the URL
df = pd.read_csv(url)

# Now `df` contains your DataFrame loaded directly from the URL
print(df.head())


# Initial data overview
print("Initial shape of the dataset:", df.shape)

# Drop completely empty columns
df_cleaned = df.dropna(axis=1, how='all')
print("Shape after dropping completely empty columns:", df_cleaned.shape)

# Columns considered critical for this analysis
critical_columns = ['YearStart', 'YearEnd', 'LocationAbbr', 'StratificationCategory1', 'Stratification1', 'DataValue']

# Dropping rows with missing values in any of the critical columns
df_cleaned = df.dropna(subset=critical_columns)

#  Provide a simple statistical summary for numerical columns
print("Statistical summary of numerical columns:")
print(df_cleaned.describe())

#save as a new csv files
cleaned_file_path = r"/content/U.S._Chronic_Disease_Indicators.csv"
df_cleaned.to_csv(cleaned_file_path, index=False)
print(f"Cleaned dataset saved to: {cleaned_file_path}")


   YearStart  YearEnd LocationAbbr LocationDesc DataSource          Topic  \
0       2019     2019           GA      Georgia      BRFSS     Disability   
1       2019     2019           GU         Guam      BRFSS      Arthritis   
2       2019     2019           GU         Guam      BRFSS   Immunization   
3       2019     2019           ME        Maine      BRFSS       Diabetes   
4       2019     2019           NV       Nevada       NVSS  Health Status   

                             Question  Response DataValueUnit  \
0          Adults with any disability       NaN             %   
1              Arthritis among adults       NaN             %   
2  Influenza vaccination among adults       NaN             %   
3               Diabetes among adults       NaN             %   
4            Life expectancy at birth       NaN         Years   

      DataValueType  ...  TopicID  QuestionID ResponseID DataValueTypeID  \
0  Crude Prevalence  ...      DIS       DIS01        NaN         CRDPR